<a href="https://colab.research.google.com/github/lolalola101/library/blob/main/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
    !pip install pandas-profiling==3.1.0  # Или попробуйте другие версии, например, 3.0.0, 2.9.0







  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 18.0 MB/s eta 0:00:00
  Created wheel for markupsafe: filename=MarkupSafe-2.0.1-cp311-cp311-linux_x86_64.whl size=27994 sha256=b7e4cf7549ec8b4634226a9fc3abd811fdeac952e9238d349cfa4188a2b99510
  Stored in directory: /root/.cache/pip/wheels/ea/18/79/6266ea508b8164a77b95aa19534c77eb805f2878612c37efca
Successfully built markupsafe
  Attempting uninstall: tangled-up-in-unicode
    Found existing installation: tangled-up-in-unicode 0.2.0
    Uninstalling tangled-up-in-unicode-0.2.0:
      Successfully uninstalled tangled-up-in-unicode-0.2.0
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.5
    Uninstalling MarkupSafe-2.1.5:
      Successfully uninstalled MarkupSafe-2.1.5
  Attempting uninstall: joblib
   

In [ ]:
import pandas as pd
import pandas_profiling
import joblib
import autoviz
from autoviz.AutoViz_Class import AutoViz

print(f"pandas version: {pd.__version__}")
print(f"pandas-profiling version: {pandas_profiling.__version__}")
print(f"joblib version: {joblib.__version__}")
#print(f"autoviz version: {autoviz.__version__}") #autoviz не имеет атрибута __version__

# Попробуйте создать простой отчет Pandas Profiling, чтобы убедиться, что все работает:
data = {'col1': [1, 2, 3], 'col2': ['a', 'b', 'c']}
df = pd.DataFrame(data)
profile = pandas_profiling.ProfileReport(df)
profile  # Отобразить отчет



PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.11/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.11/u/import-error

In [ ]:
import pandas as pd
import os
import numpy as np
from google.colab import files
from autoviz.AutoViz_Class import AutoViz
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# 1) Загрузка данных
try:
    df = pd.read_csv('Financial Indicators.csv')
    print("Данные успешно загружены из Financial Indicators.csv")
except FileNotFoundError:
    print("Файл Financial Indicators.csv не найден.  Загрузите файл в Colab или укажите правильный путь.")
    exit()

# 2) Информация о наборе данных и данные из набора
print("Информация о DataFrame:")
df.info()

print("\nПервые 5 строк DataFrame:")
print(df.head())

print("\nОписательная статистика DataFrame:")
print(df.describe())

# 3) Обработка пустых значений и дубликатов
print("\nКоличество пустых значений в каждом столбце:")
print(df.isnull().sum())

# Заполнение пустых значений (медианой для числовых столбцов)
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(df[col].median())
    else:
        print(f"Не могу заполнить пропуски в столбце '{col}'.  Требуется ручной анализ и выбор стратегии заполнения.")

print("\nКоличество пустых значений после обработки:")
print(df.isnull().sum())

print("\nКоличество дубликатов:")
print(df.duplicated().sum())

df.drop_duplicates(inplace=True)

print("\nКоличество дубликатов после удаления:")
print(df.duplicated().sum())

# 4) Конструирование признаков
df_subset = df[['Revenue', 'Net Income', 'Gross Profit', 'Total Assets', 'Total Liabilities', 'Stock Price']]

# Набор 1: Отношение чистой прибыли к выручке (Profit Margin)
df1 = df_subset.copy()
df1['ProfitMargin'] = df1['Net Income'] / df1['Revenue']

# Набор 2: Долговая нагрузка (Debt-to-Assets Ratio)
df2 = df_subset.copy()
df2['DebtToAssets'] = df2['Total Liabilities'] / df2['Total Assets']

# Набор 3:  Отношение выручки к активам (Asset Turnover Ratio)
df3 = df_subset.copy()
df3['AssetTurnover'] = df3['Revenue'] / df3['Total Assets']

# 5) Объединение данных
df_combined = df.sample(frac=0.5, random_state=42)
df_combined['random_feature'] = np.random.rand(len(df_combined))
columns_to_merge = ['Revenue', 'Net Income', 'Gross Profit']
df_merged = pd.merge(df, df_combined, how='left', on=columns_to_merge, suffixes=('_original', '_combined'))
df_merged['random_feature'].fillna(0, inplace=True)

# 6) Группировка и агрегация
aggregated_data = df.agg(
    avg_revenue=('Revenue', 'mean'),
        max_net_income=('Net Income', 'max'),
    min_stock_price=('Stock Price', 'min')
)
print("\nАгрегированные данные:\n", aggregated_data)

def range_function(series):
    return series.max() - series.min()

revenue_range = df['Revenue'].agg(range_function)
print(f"\nРазброс выручки: {revenue_range}")

# 7) Придумывание новых признаков
df['LogRevenue'] = np.log1p(df['Revenue'])
df['Revenue_NetIncome_Ratio'] = df['Revenue'] / (df['Net Income'] + 1e-9)

# 8) Составной индекс (пример)
# Замените 'Sector' и 'Industry' на реальные колонки, если они есть
# df_indexed = df.set_index(['Sector', 'Industry'])

# 9) Кодирование категориальных признаков
# Замените 'Sector' на реальную категориальную колонку, если она есть
# df = pd.get_dummies(df, columns=['Sector'], prefix=['Sector'], dummy_na=False)
# le = LabelEncoder()
# df['Industry_encoded'] = le.fit_transform(df['Industry'])

# 10) Статистические данные
print("\nОписательная статистика DataFrame:")
print(df.describe())

print("\nКорреляционная матрица:")
print(df.corr())

# 11) Визуализация с помощью Pandas
df['Stock Price'].hist(bins=30)
plt.xlabel("Цена акции")
plt.ylabel("Количество")
plt.title("Распределение цен акций")
plt.show()

df.plot.scatter(x='Revenue', y='Net Income')
plt.xlabel("Выручка")
plt.ylabel("Чистая прибыль")
plt.title("Диаграмма рассеяния: Выручка vs. Чистая прибыль")
plt.show()

df.boxplot(column='Stock Price')
plt.ylabel("Цена акции")
plt.title("Диаграмма ящиков с усиками: Цена акции")
plt.show()

# 12) Интерактивный отчет с Pandas-Profiling
try:
    profile = pandas_profiling.ProfileReport(df)
    profile.to_file("financial_data_report.html")
    files.download("financial_data_report.html")
    print("Интерактивный отчет сохранен и скачан.")
except Exception as e:
    print(f"Ошибка при создании Pandas Profiling отчета: {e}")

# 13) Визуализация с AutoViz
try:
    av = AutoViz(filename='Financial Indicators.csv', depVar=None, dfte=None, header=0, verbose=1, sep=',',
                max_rows_analyzed=150000, max_cols_analyzed=50, chart_format='png')
    viz = av.AutoViz()
except Exception as e:
    print(f"Ошибка при создании AutoViz отчета: {e}")

